# Profiling

The methods:
 - cProfile
 - line_profiler
 - dis
 - memory_profiler
 - time

A simulation program example based on which we will test different profiling techniques.

In [1]:
from dataclasses import dataclass

@dataclass
class Vec2:
    x : float
    y : float

class Object():
    def __init__(self, mass, pos:Vec2, color, vel, acc):
        self.mass = mass
        self.pos = pos
        self.vel = vel
        self.acc = acc
        self.color = color

class Ball(Object):
    def __init__(self, mass, pos, radius, color, vel=Vec2(0, 0), acc=Vec2(0, 0)):
        super().__init__(mass, pos, color, vel, acc)
        self.radius = radius
    def move(self, dis:Vec2):
        self.pos.x += dis.x
        self.pos.y += dis.y

class PhysicsEngin():
    def __init__(self):
        self.gravity = 6
    def update_pos(self, dt, obj):
        obj.vel.x += obj.acc.x * dt
        obj.vel.y += obj.acc.y * dt
        dx = obj.vel.x * dt + obj.acc.x * dt**2 / 2
        dy = obj.vel.y * dt + obj.acc.y * dt**2 / 2
        return Vec2(dx, dy)

class Simulator():
    def __init__(self, objs, engin):
        self.objs = objs
        self.engin = engin
    def simulate(self, dt):
        for obj in self.objs:
            dis = self.engin.update_pos(dt, obj)
            obj.move(dis)
    def get_objs(self):
        return self.objs

In [2]:
import pygame
import sys

def visualize(simulator):
    pygame.init() 
    
    # CREATING CANVAS 
    canvas = pygame.display.set_mode((800, 800)) 
    
    # TITLE OF CANVAS 
    pygame.display.set_caption("My Board") 
    exit = False

    # Game Loop
    while not exit: 
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT: 
                exit = True
        
        canvas.fill((0,0,0))
        simulator.simulate(0.1)
        for obj in simulator.get_objs():
            pygame.draw.circle(canvas, obj.color, (int(obj.pos.x), int(obj.pos.y)), obj.radius, 40)  # draw circle at current position
        
        if event.type == pygame.QUIT:
                pygame.display.quit()
                pygame.quit()
                sys.exit()
        
        pygame.display.flip() 

pygame 2.6.1 (SDL 2.28.4, Python 3.13.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# create ball
ball = Ball(1, Vec2(50, 50), 20, (255,0,0))
ball.vel = Vec2(50, -30)
ball.acc = Vec2(0, 10)
physics = PhysicsEngin()
simulator = Simulator([ball]*1, physics)
visualize(simulator)

SystemExit: 

/home/niuniu/Documents/DL_pytorch/high_performance_python/env/lib64/python3.13/site-packages/IPython/core/interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Profiling

### cProfile

1. Install

It is normally included in the Python's standard library.

2. Usage

The cProfile module can be used in three different ways:
 - From the command line
 ```python
 # used on the targetted program file
 python -m cProfile program.py

 # sort results
 python -m cProfile -s tottime program.py

 # output to a file
 python -m cProfile -o profile.out program.py
 ```
 - As a Python module
 ```python
 # used on a function
 import cProfile
 cProfile.run("function()")

 # wrap a code section
 pr = cProfile.Profile()
 pr.enable()
 benchmark()
 pr.disable()
 pr.print_stats()
 ```
 - With IPython
 ```bash
 # used on a function
 In [1]: %prun function()
 ```
3. output

 - ncalls: The number of times the function was called.
 - tottime: The total time spent in the function without taking into account the
calls to other functions.
 - cumtime: The time in the function including other function calls.
 - percall: The time spent for a single call of the function--it can be obtained by
dividing the total or cumulative time by the number of calls.
 - filename:lineno: The filename and corresponding line numbers. This
information is not available when calling C extensions modules.

4. display

The output can be displayed in various ways:
 - text format
 - graph format
   - KCachegrind: 
   - Gprof2Dot: https://github.com/jrfonseca/gprof2dot


In [14]:
def factorial(n):
    if n == 0:
        return 1.0
    else:
        return n * factorial(n-1)


def taylor_exp(n):
    return [1.0/factorial(i) for i in range(n)]

def taylor_sin(n):
    res = []
    for i in range(n):
        if i % 2 == 1:
            res.append((-1)**((i-1)/2)/float(factorial(i)))
        else:
            res.append(0.0)
    return res

def benchmark():
    taylor_exp(500)
    taylor_sin(500)

In [12]:
# module call
import cProfile
cProfile.run("benchmark()")

         188978 function calls (1724 primitive calls) in 0.121 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
188000/750    0.118    0.000    0.118    0.000 1056927676.py:1(factorial)
        1    0.001    0.001    0.042    0.042 1056927676.py:11(taylor_sin)
        1    0.000    0.000    0.069    0.069 1056927676.py:20(benchmark)
        1    0.000    0.000    0.027    0.027 1056927676.py:8(taylor_exp)
        2    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
        1    0.000    0.000    0.069    0.069 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 asyncio.py:225(add_callback)
        3    0.000    0.000    0.000    0.000 attrsettr.py:43(__getattr__)
        3    0.000    0.000    0.000    0.000 attrsettr.py:66(_get_attr_opt)
        1    0.000    0.000    0.016    0.016 base_eve

In [35]:
# python call
%prun benchmark()

         188978 function calls (1724 primitive calls) in 0.117 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
188000/750    0.114    0.000    0.125    0.000 1056927676.py:1(factorial)
        1    0.001    0.001    0.035    0.035 1056927676.py:11(taylor_sin)
       13    0.000    0.000    0.016    0.001 socket.py:635(send)
      504    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        2    0.000    0.000    0.038    0.019 {method '__exit__' of 'sqlite3.Connection' objects}
        1    0.000    0.000    0.024    0.024 1056927676.py:8(taylor_exp)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       54    0.000    0.000    0.000    0.000 enum.py:1585(_get_value)
  113/109    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
       13    0.000    0.000    0.000    0.000 enum.py:1592(__or__)
        3    0.000    0.000    0.000    

### line_profile

1. Install

https://github.com/rkern/line_profiler

2. Usage

 - Command: Use a decorator '@profile' and the command 'kernprof.py'
 ```bash
  $ kernprof.py -l -v program.py
 ```

 - IPython: Use the magic command 'lprun'
 ```bash
  # load the line_profiler extension, called only once
  %load_ext line_profiler

  # run the benchmark
  %lprun -f line_func_name function
 ```

In [4]:
# use IPython
%load_ext line_profiler

In [15]:
# run profile
%lprun -f taylor_exp benchmark()

Timer unit: 1e-09 s

Total time: 0.0889663 s
File: /tmp/ipykernel_3617672/1056927676.py
Function: taylor_exp at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def taylor_exp(n):
     9       501   88966265.0 177577.4    100.0      return [1.0/factorial(i) for i in range(n)]

In [31]:
# run on the example program
%lprun -f Simulator.simulate visualize(simulator)

*** SystemExit exception caught in code being profiled.

Timer unit: 1e-09 s

Total time: 0.0230864 s
File: /tmp/ipykernel_3641964/3309637629.py
Function: simulate at line 38

Line #      Hits         Time  Per Hit   % Time  Line Contents
    38                                               def simulate(self, dt):
    39      1154    4414862.0   3825.7     19.1          for obj in self.objs:
    40       577   16046739.0  27810.6     69.5              dis = self.engin.update_pos(dt, obj)
    41       577    2624820.0   4549.1     11.4              obj.move(dis)

## dis

It allows to inspect the code by th bytecode instructions.

In [17]:
import dis
dis.dis(PhysicsEngin.update_pos)

 27           RESUME                   0

 28           LOAD_FAST                2 (obj)
              LOAD_ATTR                0 (vel)
              COPY                     1
              LOAD_ATTR                2 (x)
              LOAD_FAST                2 (obj)
              LOAD_ATTR                4 (acc)
              LOAD_ATTR                2 (x)
              LOAD_FAST                1 (dt)
              BINARY_OP                5 (*)
              BINARY_OP               13 (+=)
              SWAP                     2
              STORE_ATTR               1 (x)

 29           LOAD_FAST                2 (obj)
              LOAD_ATTR                0 (vel)
              COPY                     1
              LOAD_ATTR                6 (y)
              LOAD_FAST                2 (obj)
              LOAD_ATTR                4 (acc)
              LOAD_ATTR                6 (y)
              LOAD_FAST                1 (dt)
              BINARY_OP                5 (*)
     

## Profiling Memory

## memory_profiler

1. Install

https://github.com/giampaolo/psutil

It is also included in the Python's standard library.

2. Usage

The memory_profiler module can be used in three different ways:
 - From the command line
 ```python
 # used on the targetted program file
 python -m memory_profiler program.py

 # sort results
 python -m memory_profiler -s peak -o profile.out program.py

 # output to a file
 python -m memory_profiler -o profile.out program.py
 ```
 - As a Python module
 ```python
 # used on a function
 import memory_profiler
@memory_profiler.profile
def function():
    pass

In [7]:
# as example, we created 100000 balls

def benchmark_memory():
    balls = [Ball(1, Vec2(50, 50), 20, (255,0,0), vel=Vec2(50, -30), acc=Vec2(0, 10)) for i in range(1000)]
    physics = PhysicsEngin()
    simulator = Simulator(balls, physics)
    visualize(simulator)

In [13]:
%tb benchmark_memory()

/home/niuniu/Documents/DL_pytorch/high_performance_python/env/lib64/python3.13/site-packages/IPython/core/magics/execution.py:521: UserWarning: Error changing user exception modes.
Unrecognized mode in FormattedTB: <Benchmark_memory()>
Valid modes: ['Plain', 'Context', 'Verbose', 'Minimal', 'Docs']
  warn('Error changing %s exception modes.\n%s' %


In [16]:
%load_ext memory_profiler

In [30]:
%mprun -T /tmp/test_mprun.txt -f benchmark_memory benchmark_memory()

UsageError: Could not find function 'benchmark_memory'.
NameError: name 'benchmark_memory' is not defined


In [18]:
%mprun -f taylor_exp -f benchmark()

ERROR: Could not find file /tmp/ipykernel_3641964/1056927676.py



In [29]:
%mprun -T /tmp/test_mprun.txt -f taylor_exp -f benchmark()
%cat /tmp/test_mprun.txt

ERROR: Could not find file /tmp/ipykernel_3641964/1056927676.py


*** Profile printout saved to text file /tmp/test_mprun.txt. 


In [28]:
!cat /tmp/test_mprun.txt

In [19]:
%memit benchmark()

peak memory: 140.31 MiB, increment: 0.12 MiB


### Timing

1. code

In [13]:
# code 
import time

start = time.perf_counter()
time.sleep(1)
end = time.perf_counter()
print(end - start)

1.0004313539830036


In [18]:
# decorator
def timing(func):
    def wrapper(*args, **kwargs):
        s = time.perf_counter()
        func(*args, **kwargs)
        e = time.perf_counter()
        print(f"{func.__name__} takes {e-s} seconds.")
        return func
    return wrapper

@timing
def benchmark():
    time.sleep(1)
benchmark()

benchmark takes 1.0002695790026337 seconds.


<function __main__.benchmark()>

In [28]:
# context
from dataclasses import dataclass
@dataclass
class Timer:
    text: str = "Elapsed time: {:0.4f} seconds"

    def start(self) -> None:
        self._start_time = time.perf_counter()
    def stop(self) -> float:
        # Calculate elapsed time
        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(self.text.format(elapsed_time))
        return elapsed_time
    def __enter__(self):
        self.start()
        return self
    def __exit__(self, *exc_info):
        self.stop()

with Timer():
    time.sleep(1)

Elapsed time: 1.0001 seconds


In [33]:
# timeit
from timeit import timeit
print(timeit("time.sleep(1)", number=10))

10.000969755987171


### 2. CLI

In [34]:
# for snipets
!python -m timeit "time.sleep(1)"

1 loop, best of 5: 1 sec per loop


### 3. ipython

In [12]:
# use magic command
%time time.sleep(1)

CPU times: user 1.72 ms, sys: 1.43 ms, total: 3.15 ms
Wall time: 1 s


In [31]:
%timeit time.sleep(1)

1 s ± 21.4 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
